In [3]:
import pandas as pd

df1 = pd.read_csv("df_final_web_data_pt_1.txt", sep=",")
df2 = pd.read_csv("df_final_web_data_pt_2.txt", sep=",")

In [4]:
df1.head()
df2.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58


In [5]:
# Combine both parts into a single DataFrame
df_web = pd.concat([df1, df2], ignore_index=True)

# Convert date_time to proper datetime dtype
df_web["date_time"] = pd.to_datetime(df_web["date_time"])

# Sort rows by visit_id and timestamp to keep the process sequence intact
df_web.sort_values(by=["visit_id", "date_time"], inplace=True)

In [6]:
# Remove exact duplicates (all columns identical)
df_web.drop_duplicates(inplace=True)

# Compute the step-to-step time difference within each visit
df_web["time_diff"] = df_web.groupby("visit_id")["date_time"].diff()

# Map textual process_step labels to an ordered integer column
step_order_map = {"start": 0, "step_1": 1, "step_2": 2, "step_3": 3, "confirm": 4}
df_web["step_order"] = df_web["process_step"].map(step_order_map)

# Calculate total journey time per visit (max – min timestamp)
journey_stats = (
    df_web.groupby("visit_id")["date_time"]
    .agg(["min", "max"])
    .assign(duration=lambda x: x["max"] - x["min"])
)

In [7]:
# Display quick sanity checks
print("Missing values per column:")
print(df_web.isnull().sum())
print("\nFirst five rows:")
display(df_web.head())
print("\nJourney duration sample:")
display(journey_stats.head())

Missing values per column:
client_id            0
visitor_id           0
visit_id             0
process_step         0
date_time            0
time_diff       158095
step_order           0
dtype: int64

First five rows:


,client_id,visitor_id,visit_id,process_step,date_time,time_diff,step_order
108614,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,NaT,4
108613,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,0 days 00:00:52,4
447951,9056452,306992881_89423906595,1000165_4190026492_760066,start,2017-06-04 01:07:29,NaT,0
447950,9056452,306992881_89423906595,1000165_4190026492_760066,step_1,2017-06-04 01:07:32,0 days 00:00:03,1
447949,9056452,306992881_89423906595,1000165_4190026492_760066,step_2,2017-06-04 01:07:56,0 days 00:00:24,2



Journey duration sample:


,min,max,duration
visit_id,,,
100012776_37918976071_457913,2017-04-26 13:22:17,2017-04-26 13:23:09,0 days 00:00:52
1000165_4190026492_760066,2017-06-04 01:07:29,2017-06-04 01:09:50,0 days 00:02:21
100019538_17884295066_43909,2017-04-09 16:20:56,2017-04-09 16:24:58,0 days 00:04:02
100022086_87870757897_149620,2017-05-23 20:44:01,2017-05-23 20:47:01,0 days 00:03:00
100030127_47967100085_936361,2017-03-22 11:07:49,2017-03-22 11:07:49,0 days 00:00:00


In [8]:
df_web['step_diff'] = df_web.groupby('visit_id')['step_order'].diff()
df_web[['visit_id', 'process_step', 'step_order', 'step_diff']].head(10)

,visit_id,process_step,step_order,step_diff
108614,100012776_37918976071_457913,confirm,4,NaN
108613,100012776_37918976071_457913,confirm,4,0.0
447951,1000165_4190026492_760066,start,0,NaN
447950,1000165_4190026492_760066,step_1,1,1.0
447949,1000165_4190026492_760066,step_2,2,1.0
447948,1000165_4190026492_760066,step_3,3,1.0
447947,1000165_4190026492_760066,confirm,4,1.0
240562,100019538_17884295066_43909,start,0,NaN
240561,100019538_17884295066_43909,step_1,1,1.0
240560,100019538_17884295066_43909,step_2,2,1.0


In [9]:
# Keep only forward steps (step_diff > 0)
df_web = df_web[df_web['step_diff'] > 0]
df_web.reset_index(drop=True, inplace=True)

# Preview the cleaned dataset
df_web[['visit_id', 'process_step', 'step_order', 'step_diff']].head(10)

,visit_id,process_step,step_order,step_diff
0,1000165_4190026492_760066,step_1,1,1.0
1,1000165_4190026492_760066,step_2,2,1.0
2,1000165_4190026492_760066,step_3,3,1.0
3,1000165_4190026492_760066,confirm,4,1.0
4,100019538_17884295066_43909,step_1,1,1.0
5,100019538_17884295066_43909,step_2,2,1.0
6,100019538_17884295066_43909,step_1,1,1.0
7,100019538_17884295066_43909,step_2,2,1.0
8,100019538_17884295066_43909,step_3,3,1.0
9,100019538_17884295066_43909,confirm,4,1.0


In [10]:
# Mark each visit_id as completed if it contains a 'confirm' step
df_web['completed'] = df_web['visit_id'].isin(
    df_web[df_web['process_step'] == 'confirm']['visit_id']
)
df_web[['visit_id', 'process_step', 'completed']].head(10)

,visit_id,process_step,completed
0,1000165_4190026492_760066,step_1,True
1,1000165_4190026492_760066,step_2,True
2,1000165_4190026492_760066,step_3,True
3,1000165_4190026492_760066,confirm,True
4,100019538_17884295066_43909,step_1,True
5,100019538_17884295066_43909,step_2,True
6,100019538_17884295066_43909,step_1,True
7,100019538_17884295066_43909,step_2,True
8,100019538_17884295066_43909,step_3,True
9,100019538_17884295066_43909,confirm,True


In [11]:
# Calculate completion rate
total_visits = df_web['visit_id'].nunique()
completed_visits = df_web[df_web['process_step'] == 'confirm']['visit_id'].nunique()
completion_rate = completed_visits / total_visits
completion_rate

0.659846907887644

In [21]:
# Calculate average time spent on each step
avg_time_per_step = df_web.groupby('process_step')['time_diff'].mean().sort_index()
avg_time_per_step

process_step
confirm   0 days 00:01:50.713763339
step_1    0 days 00:00:32.035359076
step_2    0 days 00:00:35.310400120
step_3    0 days 00:01:29.026666791
Name: time_diff, dtype: timedelta64[ns]

In [23]:
# Step 1: Mark rows where the user moved backwards (considered an error)
df_web['is_error'] = df_web['step_diff'] < 0

# Step 2: Count unique visit_ids that had at least one error
error_visits = df_web[df_web['is_error']]['visit_id'].nunique()

# Step 3: Count total unique visit_ids
total_visits = df_web['visit_id'].nunique()

# Step 4: Calculate the error rate
error_rate = error_visits / total_visits
error_rate

0.0